# Day 19 notebook

The objectives of this notebook are to practice 

* adding values in log space
* computing the forward matrix for a sequence and HMM
* calculating posterior probabilities for a sequence and HMM

In [ ]:
# Modules used in this activity
import math      # for log
import functools # for reduce
import random    # for simulations

## A `HiddenMarkovModel` class

In this activity, we will continue to expand on the `HiddenMarkovModel` class that we developed in the last activity.  Specifically, we will add four new methods to this class:

1. `forward_matrix` (Problem 2)
2. `backward_matrix` (provided for you)
3. `posterior_matrix` (Problem 3)
4. `log_probability` (provided for you)

In [ ]:
class HiddenMarkovModel:
    def __init__(self, states, chars, 
                 transition_prob_matrix, initial_probs, emission_prob_matrix):
        """Initializes a HiddenMarkovModel
        
        Models represented by this class do not explicitly represent a begin state and do
        not allow for an end state.
        
        Args:
            states: a string giving the characters representing the hidden states
                of the model (1 character per state)
            chars: a string giving the set of characters possibly emitted by the
                states of the model
            transition_prob_matrix: a list of lists of probabilities representing a
                transition probability matrix. transition_prob_matrix[s][t] should equal 
                P(pi_i = t | pi_{i-1} = s). Row s is thus the conditional probability 
                distribution P(pi_i | pi_{i-1} = s). The indices in this matrix correspond 
                to the indices of the states in the states argument
            initial_probs: a list of probabilities representing the initial state 
                probabilities. Entry s of this list is P(pi_1 = s), i.e., the probability that
                the first hidden state in the chain is s.  The indices of this list correspond to the
                indices of the states in the states argument.
            emission_prob_matrix: a list of lists of probabilities representing an emission
                probability matrix.  emission_prob_matrix[s][c] should equal 
                P(X_i = c | pi_i = s), i.e., the probability of state s emitting character c. 
                Row s is thus the conditional probability distribution P(X_i | pi_i = s).
                The row indices of this matrix correspond to the indices of the states in
                the states argument.  The column indices of the matrix correspond to the 
                indices of the characters in the chars argument.
        
        """
        self.states = states
        self.chars = chars
        self.transition_prob_matrix = transition_prob_matrix
        self.initial_probs = initial_probs
        self.emission_prob_matrix = emission_prob_matrix
        
        # Precompute log-transformations of the model parameters
        # to avoid computing these many times
        self.log_transition_prob_matrix = log_transform_matrix(transition_prob_matrix)
        self.log_initial_probs = log_transform_vector(initial_probs)
        self.log_emission_prob_matrix = log_transform_matrix(emission_prob_matrix)
    
    def encode_states(self, state_sequence):
        """Encodes a string of state characters as a list of indices of the states."""
        return [self.states.index(char) for char in state_sequence]

    def decode_states(self, indices):
        """Decodes a sequence of state indices into a string of the state characters."""
        return "".join(self.states[index] for index in indices)

    def encode_sequence(self, sequence):
        """Encodes a string of observed characters as a list of indices of the characters."""
        return [self.chars.index(char) for char in sequence]

    def decode_sequence(self, indices):
        """Decodes a sequence of observed character indices into a string of characters."""
        return "".join(self.chars[index] for index in indices)

    def simulate(self, length):
        """Simulates a sequence of hidden states and emitted characters of
        the given length from this HMM.
        
        Args:
            length: the length of the sequence to simulate
        Returns:
            A tuple of the form (hidden_state_string, char_string) where hidden_state_string is a
            string of state characters and char_string is a string of observed characters.
        """
        state_indices = [None] * length
        char_indices = [None] * length
        for i in range(length):
            state_probs = self.transition_prob_matrix[state_indices[i - 1]] if i > 0 else self.initial_probs
            state_indices[i] = sample_categorical(state_probs)
            char_indices[i] = sample_categorical(self.emission_prob_matrix[state_indices[i]])
            
        return (self.decode_states(state_indices), self.decode_sequence(char_indices))
        
    def log_joint_probability(self, hidden_state_string, char_string):
        """Calculates the (natural) log joint probability of a path of hidden states
        and an observed sequence given this HMM.
        
        Args:
            hidden_state_string: a string representing the sequence of hidden states (pi)
            char_string: a string representing the sequence of observed characters (X)
        Returns:
            log(P(hidden_states, observed_chars))
        """
        state_indices = self.encode_states(hidden_state_string)
        char_indices = self.encode_sequence(char_string)

        log_p = 0.0
        last_state_index = None
        for state_index, char_index in zip(state_indices, char_indices):
            if last_state_index is None:
                log_p += self.log_initial_probs[state_index]
            else:
                log_p += self.log_transition_prob_matrix[last_state_index][state_index]
            log_p += self.log_emission_prob_matrix[state_index][char_index]
            last_state_index = state_index
        return log_p

    def viterbi_matrix(self, char_string):
        """Computes the (log-transformed) Viterbi dynamic programming matrix V for
        the given observed sequence.

        Args:
            char_string: a string representing the sequence of observed characters (X)
        Returns:
            A matrix (list of lists) representing the Viterbi dynamic programming matrix,
            with rows corresponding to states and columns corresponding to positions in the
            sequence.
        """
        char_indices = self.encode_sequence(char_string)
        
        # Initialize the viterbi dynamic programming matrix
        # the entry V[k][i] corresponds to the subproblem V_k(i+1)
        # where i is a 0-based index (e.g., V[k][0] corresponds to the subproblem
        # of the most probable path of the prefix of length = 1). We will not explicitly
        # represent the begin or end states.  As a result, we will not explicitly store the
        # initialization values described in the textbook and lecture.
        V = matrix(len(self.states), len(char_string))
        if not char_string: return V
        
        # initialization (first position in sequence)
        for ell in range(len(self.states)):    # loop over hidden state indices
            V[ell][0] = (self.log_initial_probs[ell] + 
                         self.log_emission_prob_matrix[ell][char_indices[0]])

        # main fill stage
        for i in range(1, len(char_string)):    # loop over positions
            for ell in range(len(self.states)): # loop over hidden state indices
                V[ell][i] = (self.log_emission_prob_matrix[ell][char_indices[i]] + 
                             max(V[k][i - 1] + self.log_transition_prob_matrix[k][ell]
                                 for k in range(len(self.states))))

        return V
    
    def viterbi_traceback(self, V):
        """Computes a most probable path given a (log) Viterbi dynamic programming matrix.
        
        Uses a traceback procedure that does not require traceback pointers.  In the case of
        ties, this traceback prefers the state with the largest index.
        
        Args:
            V: A matrix (list of lists) representing the Viterbi dynamic programming matrix
               containing log-transformed values.
        Returns:
            A string representing a most probable sequence of hidden states
        """
        L = len(V[0])               # deduce the length of the sequence from # columns in V
        if L == 0: return ""        # empty string base case
        state_indices = [None] * L  # initialize hidden state path
        
        # determine the state at the last position in a most probable path
        max_prob, max_state = max((V[k][L - 1], k) for k in range(len(self.states)))
        state_indices[L - 1] = max_state
        
        # traceback from this last state by redoing the recurrence calculation at each step
        # the emission probabilities are not included in the calculations because they are
        # irrelevant for determining the maximizing state
        for i in range(L - 1, 0, -1):
            max_prob, max_state = max((V[k][i - 1] + self.log_transition_prob_matrix[k][max_state], k)
                                      for k in range(len(self.states)))
            state_indices[i - 1] = max_state
            
        # return string representation of hidden state path
        return self.decode_states(state_indices)        

    def most_probable_path(self, char_string):
        """Computes a most probable path of hidden states for the observed sequence.

        Args:
            char_string: a string representing the sequence of observed characters (X)
        Returns:
            A string representing a most probable sequence of hidden states.
        """
        V = self.viterbi_matrix(char_string)
        return self.viterbi_traceback(V)    
    
    def forward_matrix(self, char_string):
        """Computes the (log-transformed) Forward dynamic programming matrix f for
        the given observed sequence.

        Args:
            char_string: a string representing the sequence of observed characters (X)
        Returns:
            A matrix (list of lists) representing the Forward dynamic programming matrix,
            with rows corresponding to states and columns corresponding to positions in the
            sequence.
        """
        char_indices = self.encode_sequence(char_string)
        
        # Initialize the forward dynamic programming matrix
        # the entry f[k][i] corresponds to the subproblem f_k(i+1)
        # where i is a 0-based index (e.g., f[k][0] corresponds to the subproblem
        # of the probability of the prefix of length = 1 and ending in state k). We will 
        # not explicitly represent the begin or end states.  As a result, we will not
        # explicitly store the initialization values described in the textbook and lecture.
        f = matrix(len(self.states), len(char_string))
        if not char_string: return f
        
        # initialization
        for ell in range(len(self.states)):
            f[ell][0] = (self.log_initial_probs[ell] +
                         self.log_emission_prob_matrix[ell][char_indices[0]])

        # main fill stage
        for i in range(1, len(char_string)):
            for ell in range(len(self.states)):
                ###
                ### YOUR CODE HERE
                ###

        return f
    
    def backward_matrix(self, char_string):
        """Computes the (log-transformed) Backward dynamic programming matrix f for
        the given observed sequence.

        Args:
            char_string: a string representing the sequence of observed characters (X)
        Returns:
            A matrix (list of lists) representing the Backward dynamic programming matrix,
            with rows corresponding to states and columns corresponding to positions in the
            sequence.
        """
        char_indices = self.encode_sequence(char_string)
        
        # Initialize the backward dynamic programming matrix
        # the entry b[k][i] corresponds to the subproblem b_k(i+1)
        # where i is a 0-based index. We will not explicitly represent the begin or end states.
        # As a result, the initialization at last position sets the backward probability to 1 (0 in log space).
        b = matrix(len(self.states), len(char_string))
        if not char_string: return b
        
        # initialization
        for ell in range(len(self.states)):
            b[ell][len(char_string) - 1] = 0.0

        # main fill stage
        for i in range(len(char_string) - 2, -1, -1):
            for k in range(len(self.states)):
                b[k][i] = sum_log_probs(self.log_transition_prob_matrix[k][ell] +
                                        self.log_emission_prob_matrix[ell][char_indices[i + 1]] +
                                        b[ell][i + 1]
                                        for ell in range(len(self.states)))
        return b

    def log_probability(self, char_string):
        """Calculates the (natural) log probability (log(P(char_string))) 
        of an observed sequence given this HMM"""
        f = self.forward_matrix(char_string)
        return sum_log_probs(f[k][-1] for k in range(len(self.states)))
    
    def posterior_matrix(self, char_string):
        """Computes the posterior probability matrix for the given observed sequence.

        Args:
            char_string: a string representing the sequence of observed characters (X)
        Returns:
            a matrix (list of lists) with the entry in the kth row and ith column (e.g., m[k][i]) 
            giving the posterior probability that state k emitted character i, i.e., P(pi_i = k| x)
        """
        ###
        ### YOUR CODE HERE
        ###

    def posterior_decoding_path(self, char_string):
        """Computes the posterior decoding path of hidden states for the observed sequence.

        In the case that multiple states tie for the highest posterior probability
        at a given position, the state with the highest index is chosen.
        
        Args:
            char_string: a string representing the sequence of observed characters (X)
        Returns:
            A string representing a sequence of hidden states.
        """
        p = self.posterior_matrix(char_string)
        state_indices = [max((prob, i) for i, prob in enumerate(col))[1] for col in zip(*p)]
        return self.decode_states(state_indices)            
        
def sample_categorical(distribution):
    """Randomly sample from a categorical distribution (a discrete distribution over K categories).
    
    Args:
        distribution: a list of probabilities representing a discrete distribution over K categories.
    Returns:
        The index of the category sampled.
    """
    r = random.random()
    for i, prob in enumerate(distribution):
        if r < prob:
            return i
        else:
            r -= prob
    # in case we encounter floating point issues return the last index
    return len(distribution) - 1    

def log_transform_vector(v):
    """Returns a new vector (a list) with log-transformed values"""
    return list(map(math.log, v))

def log_transform_matrix(m):
    """Returns a new matrix (a list of lists) with log-transformed values"""
    return list(map(log_transform_vector, m))

def round_matrix(m, digits=2):
    """Returns a new matrix (a list of lists) with rounded values"""
    return [round_vector(v, digits) for v in m]
    
def round_vector(v, digits=2):
    """Returns a new vector (a list) with rounded values"""
    return [round(x, digits) for x in v]

def matrix(num_rows, num_cols, initial_value=None):
    """Constructs a matrix (a list of lists)"""
    return [[initial_value] * num_cols for i in range(num_rows)]

# Using the class above, we construct an HMM for the occasionally dishonest casino example
# described in the lecture and textbook
casino_states = "FL"     # F = fair die, L = loaded die
casino_chars = "123456"  # the six sides of the die
casino_initial_probs = [0.5, 0.5]
casino_transition_prob_matrix = [
    [0.95, 0.05],
    [0.10, 0.90]
]

casino_emission_prob_matrix = [
    [ 1/6,  1/6,  1/6,  1/6,  1/6, 1/6],
    [1/10, 1/10, 1/10, 1/10, 1/10, 1/2]
]
casino_hmm = HiddenMarkovModel(casino_states, 
                               casino_chars, 
                               casino_transition_prob_matrix, 
                               casino_initial_probs,
                               casino_emission_prob_matrix)

## PROBLEM 1: Adding probabilities in log space (1 POINT)
Recall that in the last activity we needed to implement the Viterbi algorithm in terms of log probabilities instead of raw probabilities for numerical reasons.  We will need to do the same for the Forward and Backward algorithms.  One complication with working in log space for the Forward and Backward algorithms is that adding values in log space is tricky.

In section 3.6 of the textbook, a convenient equation is provided for computing the sum of probabilities in log space.  Suppose we wish to compute $\tilde{r} = \log(p + q)$ using the log probabilities $\tilde{p} = \log(p)$ and $\tilde{q} = \log(q)$.  This computation is accurately carried out using the following equation:

$\tilde{r} = \tilde{p} + \log(1+\exp(\tilde{q} - \tilde{p}))$

where $q$ is assumed to be less than $p$, without loss of generality.

You are to implement this equation in the following function.

In [ ]:
def add_log_probs(log_p, log_q):
    """Computes the sum of two probabilities in log space."""
    ###
    ### YOUR CODE HERE
    ###


The function below, which uses the `add_log_probs` function, will be used in implementing the Forward and Backward algorithms.

In [ ]:
def sum_log_probs(log_probs):
    """Computes the sum of an iterable of probabilities in log space"""
    return functools.reduce(add_log_probs, log_probs)

In [ ]:
# tests for add_log_probs
assert round(add_log_probs(math.log(0.75), math.log(0.25)), 2) == 0.0
assert round(add_log_probs(math.log(0.5), math.log(0.25)), 2) == round(math.log(0.75), 2)
assert round(add_log_probs(math.log(0.1), math.log(0.00001)), 4) == -2.3025
assert round(add_log_probs(-1000, -1000), 2) == -999.31
assert round(add_log_probs(-2000, -3000), 2) == -2000
assert round(add_log_probs(-3000, -2000), 2) == -2000

## PROBLEM 2: Forward matrix (1 POINT)

Implement the `forward_matrix` method of the `HiddenMarkovModel` class.  Note that this method should return a matrix of **log** probabilities.

In [ ]:
# tests for forward_matrix
assert round_matrix(casino_hmm.forward_matrix('6')) == [[-2.48], 
                                                        [-1.39]]
assert round_matrix(casino_hmm.forward_matrix('1')) == [[-2.48], 
                                                        [-3.0]]
assert round_matrix(casino_hmm.forward_matrix('16')) == [[-2.48, -4.27], 
                                                         [-3.0, -3.71]]
assert round_matrix(casino_hmm.forward_matrix('165')) == [[-2.48, -4.27, -5.94], 
                                                          [-3.0, -3.71, -6.08]]
assert round_matrix(casino_hmm.forward_matrix('666661111')) == [
    [-2.48, -4.05, -5.37, -6.44, -7.34, -8.18, -9.72, -11.42, -13.19],
    [-1.39, -2.17, -2.96, -3.75, -4.54, -6.95, -9.34, -11.71, -14.05]]
assert round_matrix(casino_hmm.forward_matrix('4631262516')) == [
    [-2.48, -4.27, -5.94, -7.7, -9.49, -11.3, -13.05, -14.84, -16.65, -18.47],
    [-3.0, -3.71, -6.08, -8.43, -10.73, -11.35, -13.7, -16.01, -18.25, -18.81]]
assert round(casino_hmm.log_probability('1' * 1000), 2) == -1835.97
print("SUCCESS: forward_matrix passed all tests!")

## PROBLEM 3: Posterior probabilities (1 POINT)

Implement the `posterior_matrix` method of the `HiddenMarkovModel` class.  Note that this should return a matrix with **raw** (not log-transformed) probabilities.

In [ ]:
# tests for posterior_matrix
assert round_matrix(casino_hmm.posterior_matrix('6')) == [[0.25], 
                                                          [0.75]]
assert round_matrix(casino_hmm.posterior_matrix('1')) == [[0.62], 
                                                          [0.38]]
assert round_matrix(casino_hmm.posterior_matrix('16')) == [[0.4, 0.36], 
                                                           [0.6, 0.64]]
assert round_matrix(casino_hmm.posterior_matrix('165')) == [[0.48, 0.47, 0.54], 
                                                            [0.52, 0.53, 0.46]]
assert round_matrix(casino_hmm.posterior_matrix('666661111')) == [
    [0.04, 0.03, 0.04, 0.07, 0.16, 0.44, 0.6, 0.67, 0.7],
    [0.96, 0.97, 0.96, 0.93, 0.84, 0.56, 0.4, 0.33, 0.3]]
assert round_matrix(casino_hmm.posterior_matrix('4631262516')) == [
    [0.56, 0.55, 0.64, 0.68, 0.68, 0.64, 0.69, 0.69, 0.66, 0.58],
    [0.44, 0.45, 0.36, 0.32, 0.32, 0.36, 0.31, 0.31, 0.34, 0.42]]
print("SUCCESS: posterior_matrix passed all tests!")

### Exploration activity: how does posterior decoding compare to Viterbi?

Now that you have successfully implemented the Forward algorithm and the computation of posterior probabilties, try some simluations to see how posterior decoding compares to the Viterbi algorithm.  The method `posterior_decoding_path` is provided for you to perform posterior decoding, once you have successfully implemented the `posterior_matrix` method.

Where do the two approaches differ?  Which approach has higher accuracy?

In [ ]:
###
### YOUR CODE HERE
###


In [ ]:
###
### YOUR CODE HERE
###


###
### Your thoughts here
###
